In [ ]:
from fyp.twitter_api import twitter_api, convert_datetime_to_ISO_8601
from fyp.secrets import SECRETS

from datetime import datetime


In [ ]:
headers = {"Authorization": f"Bearer {SECRETS.TWITTER_BEARER_TOKEN}"}
start_time = convert_datetime_to_ISO_8601(datetime(2020, 6, 10, 0, 0, 0, 0))
end_time = convert_datetime_to_ISO_8601(datetime(2020, 6, 15, 23, 59, 59, 999999))


In [ ]:
class RateLimitException(Exception):
    pass


In [ ]:
# Search all tweets
# https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-all

query = (
    '("trans" OR "enby" OR "transgender" OR "nonbinary") '
    + '-"eng trans" -"#transporn" -"#porn" '
    + "-is:nullcast lang:en -is:retweet is:reply"
)

url = "https://api.twitter.com/2/tweets/search/all"

params = {
    "query": query,
    "tweet.fields": "public_metrics",
    "expansions": "author_id",
    "max_results": 10,
    "start_time": start_time,
    "end_time": end_time,
}

data_one, limit_remaining_requests, limit_reset_time = twitter_api(headers=headers, url=url, params=params, data_location='data')

if 'data' not in data_one and data_one['fyp']['error'] is True:
    print(data_one)
    if 'status' in data_one and data_one['status'] == 429:
        raise RateLimitException
    else:
        raise Exception

test_author_id = data_one['data'][0]['author_id']

params["next_token"] = data_one['meta']['next_token']
data_two, limit_remaining_requests, limit_reset_time = twitter_api(headers=headers, url=url, params=params, data_location='data')


In [ ]:
# User tweets
# https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-tweets

url = f"https://api.twitter.com/2/users/{test_author_id}/tweets"
params = {
    "exclude": 'retweets,replies',
    "expansions": 'author_id,referenced_tweets.id,entities.mentions.username',
    "max_results": 10,
    "media.fields": 'public_metrics',
    "tweet.fields": 'public_metrics,referenced_tweets',
    "user.fields": 'public_metrics,description'
}

data_three, limit_remaining_requests, limit_reset_time = twitter_api(headers=headers, url=url, params=params, data_location='data')

if 'data' not in data_three and data_three['fyp']['error'] is True:
    print(data_three)
    raise Exception


In [ ]:
# Retweeters of tweet
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/post-and-engage/api-reference/get-statuses-retweeters-ids

url = "https://api.twitter.com/1.1/statuses/retweeters/ids.json"
params = {
    'id': 1490394303562231811,
    'count': 100
}

data_four, limit_remaining_requests, limit_reset_time = twitter_api(headers=headers, url=url, params=params, data_location='ids')

if 'ids' not in data_four and data_four['fyp']['error'] is True:
    print(data_four)
    raise Exception


In [ ]:
# Get user mentions
# https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-mentions

url = f"https://api.twitter.com/2/users/{test_author_id}/mentions"
params = {
    "max_results": 10,
    "expansions": 'author_id'
}

data_five, limit_remaining_requests, limit_reset_time = twitter_api(headers=headers, url=url, params=params, data_location='data')

if 'data' not in data_five and data_five['fyp']['error'] is True:
    print(data_five)
    raise Exception


In [ ]:
# Get follower ids
# https://developer.twitter.com/en/docs/twitter-api/v1/accounts-and-users/follow-search-get-users/api-reference/get-followers-ids

url = "https://api.twitter.com/1.1/followers/ids.json"
params = {
    "user_id": test_author_id,
    "count": 10
}

data_six, limit_remaining_requests, limit_reset_time = twitter_api(headers=headers, url=url, params=params, data_location='ids')

if 'ids' not in data_six and data_six['fyp']['error'] is True:
    print(data_six)
    raise Exception


In [ ]:
# Get user details
# https://developer.twitter.com/en/docs/twitter-api/users/lookup/api-reference/get-users

url = "https://api.twitter.com/2/users"
params = {
    "ids": f"{data_one['data'][0]['author_id']},{data_one['data'][1]['author_id']}",
    "user.fields": "public_metrics"
}

data_seven, limit_remaining_requests, limit_reset_time = twitter_api(headers=headers, url=url, params=params, data_location='data')

if 'data' not in data_seven and data_seven['fyp']['error'] is True:
    print(data_seven)
    raise Exception
